In [12]:
from utils import get_puzzle_input_as_rows
from typing import List, Dict

In [26]:
from pprint import pprint
from dataclasses import dataclass
from typing import Dict, Optional

In [49]:
Name = str

@dataclass
class FileSystem():
    children: "FSObject"

@dataclass
class FSObject():
    name: Name
    children: Optional[Dict[Name, "FSObject"]]
    size: Optional[int]
    type: str

def get_file_system_object(keys: List, fs: FileSystem):
    fs_object = fs
    for k in keys:
        fs_object = fs_object.children[k]
    return fs_object

In [86]:
def construct_file_system(rows):

    root = FSObject(name="/", children={}, size=None, type="dir")
    file_system = FileSystem(children={"/": root})

    pwd = []
    for row in rows:

        current_item = get_file_system_object(pwd, file_system)

        if row.startswith("$ cd"):
            _, command = row.split("$ cd ")
            if command == "/":
                pwd = ["/"]
            elif command == "..":
                pwd.pop()
            else:
                pwd.append(command)
        elif row.startswith("$ ls"):
            pass  # was a list call - do nothing
        else:
            if row.startswith("dir "):
                _, dir = row.split("dir ")
                current_item.children[dir] = FSObject(name=dir, children={}, size=None, type="dir")
            else:
                size, file_name = row.split(" ")
                current_item.children[file_name] = FSObject(name=file_name, children=None, size=int(size), type="file")

    return file_system

def calculate_sizes(fs_object: FSObject):
    if fs_object.type == "file":
        return fs_object.size
    else:  # a dir
        fs_object.size = sum(calculate_sizes(o) for o in fs_object.children.values())
        return fs_object.size

def get_dirs_up_to_max(fs_object: FSObject, max_size: int):
    if fs_object.type == "file":
        return
    if fs_object.size <= max_size:
        yield fs_object.size
    for child in fs_object.children.values():
        yield from get_dirs_up_to_max(child, max_size)

In [90]:
def question_7_a():
    rows = get_puzzle_input_as_rows(2022, 7)
    file_system = construct_file_system(rows)
    root = file_system.children["/"]
    calculate_sizes(root)
    print(sum(get_dirs_up_to_max(root, 100000)))

question_7_a()  # 2104783

2104783
